# DS 22/23 2A Project

### Load downloaded Spotify Streaming data
The data is split up in multiple JSON files. We need to all read in all files and merge them into one big DataFrame for further processing.

In [4]:
# Load raw streaming history data
import os
import pandas as pd

DATA_PATH = './data/'

# List of all relevant files
file_list = [
    DATA_PATH + filename for filename in os.listdir(DATA_PATH) if filename != '.gitkeep']

# List of Dataframes for each file
df_list = [pd.read_json(file) for file in file_list]

# Concatenating all df's together
df = pd.concat(df_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174902 entries, 0 to 16487
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ts                                 174902 non-null  object 
 1   username                           174902 non-null  object 
 2   platform                           174902 non-null  object 
 3   ms_played                          174902 non-null  int64  
 4   conn_country                       174902 non-null  object 
 5   ip_addr_decrypted                  174902 non-null  object 
 6   user_agent_decrypted               172199 non-null  object 
 7   master_metadata_track_name         166423 non-null  object 
 8   master_metadata_album_artist_name  166423 non-null  object 
 9   master_metadata_album_album_name   166423 non-null  object 
 10  spotify_track_uri                  166423 non-null  object 
 11  episode_name                       7856 

## Setup Database and insatntiate engine
The util functions help us to create all required tables for the designed star schema. The used database credentials can be set in the .env file via the following variables:
- DB_DRIVER
- DB_NAME
- DB_USERNAME
- DB_PASSWORD
- DB_SERVER
- DB_PORT

In [5]:
from database import get_sql_engine, create_tables

create_tables()
engine = get_sql_engine()

## Setup Spotify API connection
We make use of the spotipy package to interact with the public spotify web api. This is required to enrich the streaming history data with information on songs, genres, artist and musical features. To initiialize the connection the credentials must be set in the .env file via the variables:
- SPOTIFY_CLIENT_ID
- SPOTIFY_CLIENT_SECRET

In [6]:
# Spotify API Integration
import os

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

CLIENT_ID = os.environ['SPOTIFY_CLIENT_ID']
CLIENT_SECRET = os.environ['SPOTIFY_CLIENT_SECRET']

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET), retries=5, backoff_factor=0.5)

## Song Data preprocessing

In [7]:
song_df = df[['master_metadata_track_name', 'spotify_track_uri']]
song_df = song_df.dropna()
song_df = song_df.drop_duplicates()
song_df = song_df.rename(columns={
    'master_metadata_track_name': 'name',
    'spotify_track_uri': 'spotify_id',
})
song_df = song_df.reset_index(drop=True)

### Song data enrichment

In [8]:
from tqdm.auto import tqdm

chunk_size = 50

for n in tqdm(range(0, len(song_df), chunk_size)):
    # Prepare all Spotify internal Song IDs of the chunk for querrying
    song_ids = song_df.loc[n:n+chunk_size-1, 'spotify_id'].tolist()
    
    # Query Spotify API
    try:
        tracks_result = sp.tracks(song_ids)
        audio_features_result = sp.audio_features(song_ids)
    except spotipy.client.SpotifyException as e:
        print("Spotipy Exception: "+e)
        continue

    tracks = tracks_result['tracks']
    features = audio_features_result

    artist_names = [track['artists'][0]['name']
                    if track is not None else None for track in tracks]
    artist_uris = [track['artists'][0]['uri']
                   if track is not None else None for track in tracks]
    track_uris = [track['uri']
                  if track is not None else None for track in tracks]
    track_durations = [track['duration_ms']
                       if track is not None else None for track in tracks]
    track_explicit = [track['explicit']
                      if track is not None else None for track in tracks]
    track_popularities = [track['popularity']
                          if track is not None else None for track in tracks]
    features_key = [feat['key']
                    if feat is not None else None for feat in features]
    features_mode = [feat['mode']
                     if feat is not None else None for feat in features]
    features_danceability = [feat['danceability']
                             if feat is not None else None for feat in features]
    features_energy = [feat['energy']
                       if feat is not None else None for feat in features]
    features_loudness = [feat['loudness']
                         if feat is not None else None for feat in features]
    features_speechiness = [feat['speechiness']
                            if feat is not None else None for feat in features]
    features_acousticness = [feat['acousticness']
                              if feat is not None else None for feat in features]
    features_instrumentalness = [feat['instrumentalness']
                                 if feat is not None else None for feat in features]
    features_liveness = [feat['liveness']
                         if feat is not None else None for feat in features]
    features_valence = [feat['valence']
                        if feat is not None else None for feat in features]
    features_tempo = [feat['tempo']
                      if feat is not None else None for feat in features]

    song_df.loc[n:n+chunk_size-1, 'artist_name'] = artist_names
    song_df.loc[n:n+chunk_size-1, 'artist_uri'] = artist_uris
    song_df.loc[n:n+chunk_size-1, 'uri'] = track_uris
    song_df.loc[n:n+chunk_size-1, 'duration'] = track_durations
    song_df.loc[n:n+chunk_size-1, 'explicitness'] = track_explicit
    song_df.loc[n:n+chunk_size-1, 'popularity'] = track_popularities
    song_df.loc[n:n+chunk_size-1, 'key'] = features_key
    song_df.loc[n:n+chunk_size-1, 'mode'] = features_mode
    song_df.loc[n:n+chunk_size-1, 'danceability'] = features_danceability
    song_df.loc[n:n+chunk_size-1, 'energy'] = features_energy
    song_df.loc[n:n+chunk_size-1, 'loudness'] = features_loudness
    song_df.loc[n:n+chunk_size-1, 'speechiness'] = features_speechiness
    song_df.loc[n:n+chunk_size-1, 'acousticness'] = features_acousticness
    song_df.loc[n:n+chunk_size-1, 'instrumentalness'] = features_instrumentalness
    song_df.loc[n:n+chunk_size-1, 'liveness'] = features_liveness
    song_df.loc[n:n+chunk_size-1, 'valence'] = features_valence
    song_df.loc[n:n+chunk_size-1, 'tempo'] = features_tempo

song_df


  0%|          | 0/918 [00:00<?, ?it/s]

In [ ]:
song_table_df = song_df.drop(
    columns=['artist_name', 'artist_uri', 'uri'], axis=1)

song_table_df['name'] = song_table_df['name'].str.encode(
    'latin-1', 'ignore').str.decode('latin-1')

song_table_df.to_sql('songs', engine, schema='project',
                     index=True, index_label='id', if_exists='append')


861

## Artist Data Preprocessing

In [ ]:
from ast import literal_eval

artist_df = song_df[['artist_name', 'artist_uri']]
artist_df = artist_df.drop_duplicates().dropna()
artist_df = artist_df.reset_index(drop=True)

chunk_size = 50

for n in tqdm(range(0, len(artist_df), chunk_size)):
    artists_ids = artist_df.loc[n:n+chunk_size-1, 'artist_uri'].tolist()
    try:
        artists_result = sp.artists(artists_ids)
        artists_result = artists_result['artists']
    except spotipy.client.SpotifyException as e:
        print("Spotipy Exception: "+e)
        continue

    popularities = [artist['popularity']
                    if artist is not None else None for artist in artists_result]
    genres = [str(artist['genres'])
                    if artist is not None else None for artist in artists_result]

    artist_df.loc[n:n+chunk_size-1, 'popularity'] = popularities
    artist_df.loc[n:n+chunk_size-1, 'genres'] = genres

artist_df['genres'] = artist_df['genres'].apply(literal_eval)

  0%|          | 0/285 [00:00<?, ?it/s]

In [ ]:
artist_table_df = artist_df[['artist_name', 'artist_uri', 'popularity']]
artist_table_df = artist_table_df.rename(columns={
    'artist_name': 'name',
    'artist_uri': 'spotify_id'
})
artist_table_df['name'] = artist_table_df['name'].str.encode(
    'latin-1', 'ignore').str.decode('latin-1')
artist_table_df.to_sql('artists', engine, schema='project',
                       index=True, index_label='id', if_exists='append')


228

## Genre Data Preprocessing

In [ ]:
genre_df = artist_df['genres'].explode('genres')
genre_df = genre_df.drop_duplicates().dropna().reset_index(drop=True)
genre_df.name = 'name'

genre_df.to_sql('genres', engine, schema='project',
                index=True, index_label='id', if_exists='append')

442

In [ ]:
link_df = artist_df.explode('genres')
link_df = link_df.rename(columns={'genres': 'name'})

genre_df = genre_df.to_frame()
genre_df['genre_id'] = genre_df.index

artist_genre_id_df = pd.merge(link_df, genre_df, on='name', how='left')
artist_genre_id_df

,artist_name,artist_uri,popularity,name,genre_id
0,R.I.O.,spotify:artist:0Ol3Jol2T3lZZVLNNzWPhj,62.0,dance pop,0.0
1,R.I.O.,spotify:artist:0Ol3Jol2T3lZZVLNNzWPhj,62.0,german dance,1.0
2,R.I.O.,spotify:artist:0Ol3Jol2T3lZZVLNNzWPhj,62.0,pop house,2.0
3,Clean Bandit,spotify:artist:6MDME20pz9RveH9rEXvrOM,76.0,dance pop,0.0
4,Clean Bandit,spotify:artist:6MDME20pz9RveH9rEXvrOM,76.0,edm,3.0
...,...,...,...,...,...
35037,Vluestar,spotify:artist:2FcgwIGEPmPyItFPscscDO,58.0,aesthetic rap,2074.0
35038,Vluestar,spotify:artist:2FcgwIGEPmPyItFPscscDO,58.0,lo-fi chill,453.0
35039,Vluestar,spotify:artist:2FcgwIGEPmPyItFPscscDO,58.0,sad lo-fi,454.0
35040,Vluestar,spotify:artist:2FcgwIGEPmPyItFPscscDO,58.0,sad rap,455.0


## Stream Data preprocessing

In [ ]:
stream_df = df[['ts', 'spotify_track_uri', 'shuffle',
                'skipped', 'conn_country', 'ms_played']]
stream_df = stream_df.rename(columns={'conn_country': 'country', 'ms_played': 'duration'})
stream_df = stream_df.dropna(axis=0, subset=['spotify_track_uri'])

song_id_df = song_df[['spotify_id', 'artist_uri']]
song_id_df['song_id'] = song_df.index

artist_id_df = artist_df[['artist_uri', 'genres']]
artist_id_df['artist_id'] = artist_df.index

song_id_df = song_id_df.merge(artist_id_df, on='artist_uri', how='inner')

stream_df = stream_df.merge(song_id_df, left_on='spotify_track_uri', right_on="spotify_id")
stream_df = stream_df.drop(
    columns=['spotify_track_uri', 'spotify_id', 'artist_uri'], axis=1)


/var/folders/r5/v8m2d0656qqcmqdnp93q133r0000gn/T/ipykernel_8721/2036528587.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_id_df['song_id'] = song_df.index
/var/folders/r5/v8m2d0656qqcmqdnp93q133r0000gn/T/ipykernel_8721/2036528587.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artist_id_df['artist_id'] = artist_df.index


In [ ]:
stream_date_df = stream_df[['ts']]
stream_date_df['ts'] = pd.to_datetime(stream_date_df['ts'])
stream_date_df['stream_id'] = stream_df.index

stream_date_df = stream_date_df.rename(columns={'ts': 'start_time'})
stream_date_df['year'] = stream_date_df['start_time'].dt.year
stream_date_df['month'] = stream_date_df['start_time'].dt.month
stream_date_df['day'] = stream_date_df['start_time'].dt.day
stream_date_df['hour'] = stream_date_df['start_time'].dt.hour
stream_date_df['weekday'] = stream_date_df['start_time'].dt.weekday

stream_date_df = stream_date_df.drop(columns=['stream_id'], axis=1)
stream_date_df.to_sql('dates', engine,
                      schema='project', if_exists='append', index=True, index_label='id')


/var/folders/r5/v8m2d0656qqcmqdnp93q133r0000gn/T/ipykernel_8721/1910703316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stream_date_df['ts'] = pd.to_datetime(stream_date_df['ts'])
/var/folders/r5/v8m2d0656qqcmqdnp93q133r0000gn/T/ipykernel_8721/1910703316.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stream_date_df['stream_id'] = stream_df.index


687

In [ ]:
stream_table_df = stream_df.drop(columns=['genres', 'ts'], axis=1)
stream_table_df['date_id'] = stream_date_df.index
stream_table_df['skipped'] = [True if skip ==
                              1 else False for skip in stream_table_df['skipped'].tolist()]
stream_table_df.to_sql('streams', engine, schema='project',
                 index=True, index_label='id', if_exists='append')


687

In [ ]:
stream_genre_df = stream_df[['genres']]
stream_genre_df['stream_id'] = stream_df.index

stream_genre_df = stream_genre_df.explode('genres').dropna()
stream_genre_df = stream_genre_df.rename(columns={'genres': 'name'})
stream_genre_df = stream_genre_df.merge(genre_df, on="name", how='left').drop(columns=['name'], axis=1)

stream_genre_df.to_sql('streams_genres', engine, schema='project', if_exists='append', index=False)

/var/folders/r5/v8m2d0656qqcmqdnp93q133r0000gn/T/ipykernel_8721/2813369518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stream_genre_df['stream_id'] = stream_df.index


408